### Analisis de Sentimiento a tweets en Español con el clasificador Naive Bayes

#### Tweets obtenidos de base de datos con tweets recolectados en español de usuarios con geolocalizacion en Guatemala

### tweets class
* 0 = negativo
* 1 = positivo
* 2 = neutral

### Imports:

In [1]:
import MySQLdb
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [2]:
#Retrieve tweets from db
conn = MySQLdb.connect("13.58.190.139","root","123","tesis" )
data = pd.read_sql("select * from tweets where class is not null limit 2000", conn)
data_copy = data

In [3]:
#Split label from dataset
y = data_copy["class"]
X = data_copy["text"]
#Create dummy indicators for column label
#y = pd.get_dummies(y, columns=["class"], prefix="class")
#Split dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [4]:
# Import spanish stopword
spanish_stopwords = stopwords.words('spanish')

vectorizer = CountVectorizer(
                analyzer = 'word',
                lowercase = True,
                ngram_range = (1,3),
                stop_words = spanish_stopwords)

In [5]:
# Bag of Words from training set
X_train_counts = vectorizer.fit_transform(X_train)

In [6]:
# Train classifier with TF-IDF words weigth
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [7]:
# Classifier
nv_classifier = MultinomialNB().fit(X_train_tfidf, y_train)

In [8]:
# Fit classifier with test set
X_new_counts = vectorizer.transform(X_test)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = nv_classifier.predict(X_new_tfidf)

In [9]:
#Check accuracy
np.mean(predicted == y_test)  

0.49199999999999999

In [10]:
#Decode Labels from predicted output
predict_decode = []
for value in predicted:
    if value == 0:
        predict_decode.append("Negativo")
    else:
        if value == 1:
            predict_decode.append("Positivo")
        else:
            predict_decode.append("Neutral")

In [11]:
#Remove index from Series
test_tweets = X_test.reset_index()
predicted_serie = pd.Series(predict_decode, index=None)

#Convert Series to DataFrame
df = pd.DataFrame(test_tweets, columns=['text'])
df2 = predicted_serie.to_frame(name='predicted')
df['predicted']=df2.values

In [12]:
#Display results
header_style = dict(selector="th", props=[('text-align', 'left')])
pd.set_option('display.max_colwidth',140)
df.style.set_properties(**{'text-align':'left'}).set_table_styles([header_style])

,text,predicted
0,6 teleton 2017 7 partido patriota 8 ligamx 9 delito 10 honduras 2017 7 15 08 55 cst trndnl URL,Neutral
1,xetulul un parque de diversiones guatemalteco de grandes alturas URL,Neutral
2,esperando ya el 15,Neutral
3,ineficiencia descontrol pmt villa canales pmts entorpecen el transito los alamos AT USER AT USER URL,Neutral
4,un poco de rock nacional me vendria muuuuuy bien,Neutral
5,politica con 111 diputados presentes en el hemiciclo inicia la sesion plenaria en el congreso foto URL,Neutral
6,AT USER AT USER AT USER eso decia yo solo que mejor compraran 5 o 10 radiopatrullas baratas pero funcionales,Neutral
7,AT USER es que me da tanta risa lo mulas que son esos pizados jajajaja lo meten al bote solo por chingar,Negativo
8,es que lo veo con esa guitarra y todo se me estremece por dentro que sera,Neutral
9,puta madre ignorenlo que si no pasa hablando de lo mismo toda la noche URL,Negativo
